# Exercise 2

## Part 1

First, we use sympy to generate a formula for $\phi_{s,t,n}$ which is supposed to be a tautology iff Ramsey number $R(s, t) < n$.

In mathematical notation, the formula is given by
$\phi_{s, t, n} = (\bigwedge_{P \in N_s} \bigvee_{p \in P} p) \wedge (\bigwedge_{P \in N_t} \bigvee_{p \in P} p) $
where  
 - $ N_r $ is a set of all $r$-element combinations of a set of variables $\{p_{ij}\}_{i,j \in \{1..n\}}$
 - a variable $p_{ij}$ has indicates whether there is an edge from node $i$ to node $j$ in a graph.

In [1]:
# define n, s and t
n = 4
s = 3
t = 3

In [2]:
from sympy import *
from itertools import combinations, combinations_with_replacement

In [3]:

p = [[None for _ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        p[i][j] = Symbol(f"p{i}{j}")

In [4]:
formula = And(False)

def comp_conn(s):
    formula = And(True)
    for i in s:
        for j in s:
            formula = And(formula, p[i][j])
#     print(formula)
    return formula

def comp_disc(s):
    formula = And(True)
    for i in s:
        for j in s:
            formula = And(formula, Not(p[i][j]))
#     print(formula)
    return formula

for c in combinations(list(range(n)), s):
    formula = Or(formula, comp_conn(c))
    
for c in combinations(list(range(n)), t):
    formula = Or(formula, comp_disc(c))

In [5]:
# Final formula for the given n, t, s is show below
formula_cnf = to_cnf(Not(formula))
print(formula_cnf)

(p00 | p01 | p02 | p10 | p11 | p12 | p20 | p21 | p22) & (p00 | p01 | p03 | p10 | p11 | p13 | p30 | p31 | p33) & (p00 | p02 | p03 | p20 | p22 | p23 | p30 | p32 | p33) & (p11 | p12 | p13 | p21 | p22 | p23 | p31 | p32 | p33) & (~p00 | ~p01 | ~p02 | ~p10 | ~p11 | ~p12 | ~p20 | ~p21 | ~p22) & (~p00 | ~p01 | ~p03 | ~p10 | ~p11 | ~p13 | ~p30 | ~p31 | ~p33) & (~p00 | ~p02 | ~p03 | ~p20 | ~p22 | ~p23 | ~p30 | ~p32 | ~p33) & (~p11 | ~p12 | ~p13 | ~p21 | ~p22 | ~p23 | ~p31 | ~p32 | ~p33)


## Part 2

And now, using off-the-shelf `Glucose4` SAT solver we try to find Ramsey numbers for $s, t < 7$
Timeout for one step of computation is set below.

In [35]:
timeout = 2 # seconds

In [7]:
from pysat.solvers import Glucose4
import time

def check_n(n, s, t, queue):
    def combs(k, ss=list(range(1, n+1))):
        return combinations(ss, k)

    NUMS = {tuple(p): e + 1 for e, p in enumerate(combs(2))}
    def get_number(c):
        return NUMS[tuple(c)]

    time.sleep(1)
    g = Glucose4()

    for c in combs(s):
        clause = [get_number(t) for t in combs(2, c)]
        g.add_clause(clause)

    for c in combs(t):
        clause = [-get_number(t) for t in combs(2, c)]
        g.add_clause(clause)
    
    queue.put(g.solve())
#     return g.solve()

In [8]:
from tqdm import tqdm
import multiprocessing

def run_until_timeout(f, args, timeout):
    p = multiprocessing.Process(target=f, args=args)
    
    p.start()

    p.join(timeout)

    if p.is_alive():
        p.terminate()
        p.join()

In [40]:
def run(f, s, t):
#     print(s, t)
    n = 0
    q = multiprocessing.Queue()
    res = True
    while res:
        n += 1
        run_until_timeout(f, (n, s, t, q), timeout)
#         print(f"Value for s={s}, t={t} is >={n}")
        try:
            res = q.get(True, timeout)
        except:
            # queue was empty
            print(f"Timeout: highest value for s={s}, t={t} is n={n}")
            return
        
    print(f"OK: value for s={s}, t={t} is n={n}")
    return

And here we get values (or values-at-timeouts) for various pairs $(s, t)$ such that $s \leq t \leq 7$ (by symmetry, $R(s, t) = R(t, s)$).

In [17]:
for s, t in list(combinations_with_replacement(range(1, 8), 2)):
    if s <= t:
        run(check_n, s, t)

OK: value for s=1, t=1 is n=1
OK: value for s=1, t=2 is n=1
OK: value for s=1, t=3 is n=1
OK: value for s=1, t=4 is n=1
OK: value for s=1, t=5 is n=1
OK: value for s=1, t=6 is n=1
OK: value for s=1, t=7 is n=1
OK: value for s=2, t=2 is n=2
OK: value for s=2, t=3 is n=3
OK: value for s=2, t=4 is n=4
OK: value for s=2, t=5 is n=5
OK: value for s=2, t=6 is n=6
OK: value for s=2, t=7 is n=7
OK: value for s=3, t=3 is n=6
OK: value for s=3, t=4 is n=9
Timeout: highest value for s=3, t=5 is n=14
Timeout: highest value for s=3, t=6 is n=18
Timeout: highest value for s=3, t=7 is n=21
Timeout: highest value for s=4, t=4 is n=18
Timeout: highest value for s=4, t=5 is n=24
Timeout: highest value for s=4, t=6 is n=25
Timeout: highest value for s=4, t=7 is n=21
Timeout: highest value for s=5, t=5 is n=29
Timeout: highest value for s=5, t=6 is n=24
Timeout: highest value for s=5, t=7 is n=21
Timeout: highest value for s=6, t=6 is n=23
Timeout: highest value for s=6, t=7 is n=21
Timeout: highest value

## Part 3

The same thing as before, but this time using DP and DPLL algorithms implemented in exercise 1.

In [47]:
from code_python import DP

def check_n_dp(n, s, t, queue):
    def combs(k, ss=list(range(1, n+1))):
        return combinations(ss, k)

    NUMS = {tuple(p): e + 1 for e, p in enumerate(combs(2))}
    def get_number(c):
        return NUMS[tuple(c)]

    time.sleep(1)
    g = set()

    for c in combs(s):
        clause = frozenset([get_number(t) for t in combs(2, c)])
        g.add(clause)

    for c in combs(t):
        clause = frozenset([-get_number(t) for t in combs(2, c)])
        g.add(clause)
    
    res = DP.DP(g, set(NUMS.values()))
    queue.put(res == "SAT")

In [52]:
# this time, only to s, t <= 5, the rest is all timeouts
for s, t in list(combinations_with_replacement(range(1, 6), 2)):
    if s <= t:
        run(check_n_dp, s, t)

OK: value for s=1, t=1 is n=1
OK: value for s=1, t=2 is n=1
OK: value for s=1, t=3 is n=1
OK: value for s=1, t=4 is n=1
OK: value for s=1, t=5 is n=1
OK: value for s=2, t=2 is n=2
OK: value for s=2, t=3 is n=3
OK: value for s=2, t=4 is n=4
OK: value for s=2, t=5 is n=5
OK: value for s=3, t=3 is n=6
Timeout: highest value for s=3, t=4 is n=7
Timeout: highest value for s=3, t=5 is n=8
Timeout: highest value for s=4, t=4 is n=8
Timeout: highest value for s=4, t=5 is n=9
Timeout: highest value for s=5, t=5 is n=9


In [53]:
from code_python import DPLL

def check_n_dpll(n, s, t, queue):
    def combs(k, ss=list(range(1, n+1))):
        return combinations(ss, k)

    NUMS = {tuple(p): e + 1 for e, p in enumerate(combs(2))}
    def get_number(c):
        return NUMS[tuple(c)]

    time.sleep(1)
    g = set()

    for c in combs(s):
        clause = frozenset([get_number(t) for t in combs(2, c)])
        g.add(clause)

    for c in combs(t):
        clause = frozenset([-get_number(t) for t in combs(2, c)])
        g.add(clause)
    
    res = DPLL.DPLL(g, set(NUMS.values()))
    queue.put(res == "SAT")

In [55]:
# here also just for s, t <= 5
for s, t in list(combinations_with_replacement(range(1, 6), 2)):
    if s <= t:
        run(check_n_dpll, s, t)

OK: value for s=1, t=1 is n=1
OK: value for s=1, t=2 is n=1
OK: value for s=1, t=3 is n=1
OK: value for s=1, t=4 is n=1
OK: value for s=1, t=5 is n=1
OK: value for s=2, t=2 is n=2
OK: value for s=2, t=3 is n=3
OK: value for s=2, t=4 is n=4
OK: value for s=2, t=5 is n=5
Timeout: highest value for s=3, t=3 is n=6
Timeout: highest value for s=3, t=4 is n=8
Timeout: highest value for s=3, t=5 is n=9
Timeout: highest value for s=4, t=4 is n=11
Timeout: highest value for s=4, t=5 is n=12
Timeout: highest value for s=5, t=5 is n=13
